In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd

In [4]:
%cd '/content/drive/My Drive/Colab Notebooks/codigo-ordenado'

/content/drive/My Drive/Colab Notebooks/codigo-ordenado


**LISTO LOS ARCHIVOS ORIGINALES, TAL Y CUAL COMO SON OBTENIDOS DEL INEC**

In [5]:
rutas_descargados_inec = os.path.join(os.getcwd(), 'originales-inec')
descargados_inec = os.listdir(rutas_descargados_inec)
descargados_inec

['INEC_afeCOVID_BD_2020.csv',
 'INEC_ctnac_BD_2020.csv',
 'INEC_eunac_BD_2020.csv',
 'INEC_cgnac_BD_2020.csv',
 'INEC_cpnac_BD_2020.csv',
 'AMBNAC.csv']

In [6]:
covid = pd.read_csv(os.path.join(rutas_descargados_inec, 'INEC_afeCOVID_BD_2020.csv'), sep=';')
caracteristicas = pd.read_csv(os.path.join(rutas_descargados_inec, 'INEC_cgnac_BD_2020.csv'), sep=';')
empleo = pd.read_csv(os.path.join(rutas_descargados_inec, 'INEC_eunac_BD_2020.csv'), sep=';')
permanentes = pd.read_csv(os.path.join(rutas_descargados_inec, 'INEC_cpnac_BD_2020.csv'), sep=';')
transitorios = pd.read_csv(os.path.join(rutas_descargados_inec, 'INEC_ctnac_BD_2020.csv'), sep=';')
ambnac = pd.read_csv(os.path.join(rutas_descargados_inec, 'AMBNAC.csv'), sep=',')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (74) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## **LIMPIAR EL ARCHIVO Y CONVERTIR A DUMMIES LAS QUE CORRESPONDEN**

***funciones***

In [7]:
def limpieza_archivo_original(df):
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
  df = df.replace('', np.NaN)
  print('Se han quitado los espacios en blanco')
  df = df.applymap(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
  print('Se ha establecido el punto como separador de decimales')
  return df

def columnas_dummies(df):
  modificar = []
  for column in df.columns:
    if df[column].nunique() == 2:
      modificar.append(column)
  return modificar

def conversor_dummy(elemento):
  if elemento in ['2', 2, 2.0, '2.0']:
    return 0
  elif elemento in ['1', 1, 1.0, '1.0']:
    return 1
  else:
    return elemento

def convertir_dummies(df, columnas, tipo=str):
  for columna in columnas:
    df[columna] = df[columna].astype(tipo)
    df[columna] = df[columna].apply(conversor_dummy)
  df = df.applymap(lambda x: np.NaN if x == 'nan' else x)
  return df

**LIMPIAMOS AMBNAC**

In [9]:
ambnac = ambnac[['identificador', 'pa_arealm', 'pa_areempa', 'eq_earado', 'eq_semman', 'eq_emotoguadana', 'eq_ebmanual',	'eq_ebestaciona',	'eq_ecosgranfigr']]
ambnac.sort_values('identificador', inplace=True)
ambnac

,identificador,pa_arealm,pa_areempa,eq_earado,eq_semman,eq_emotoguadana,eq_ebmanual,eq_ebestaciona,eq_ecosgranfigr
0,1015101086210001,2,2,NaN,2.0,2,2,2,2.0
1,1015101086210002,2,2,NaN,2.0,2,2,2,2.0
2,1015101086210003,2,2,NaN,2.0,2,2,2,2.0
3,1015101086210004,2,2,NaN,2.0,2,2,2,2.0
4,1015101086210005,2,2,NaN,2.0,2,2,2,2.0
...,...,...,...,...,...,...,...,...,...
26521,24015504000390009,2,2,NaN,2.0,1,1,2,2.0
26522,24015504000390010,2,2,NaN,1.0,1,1,2,2.0
26523,24015504000390011,2,2,NaN,1.0,1,1,2,2.0
26524,24015504000390012,2,2,NaN,1.0,1,2,1,2.0


In [10]:
ambnac = limpieza_archivo_original(ambnac)
ambnac

Se han quitado los espacios en blanco
Se ha establecido el punto como separador de decimales


,identificador,pa_arealm,pa_areempa,eq_earado,eq_semman,eq_emotoguadana,eq_ebmanual,eq_ebestaciona,eq_ecosgranfigr
0,1015101086210001,2,2,NaN,2.0,2,2,2,2.0
1,1015101086210002,2,2,NaN,2.0,2,2,2,2.0
2,1015101086210003,2,2,NaN,2.0,2,2,2,2.0
3,1015101086210004,2,2,NaN,2.0,2,2,2,2.0
4,1015101086210005,2,2,NaN,2.0,2,2,2,2.0
...,...,...,...,...,...,...,...,...,...
26521,24015504000390009,2,2,NaN,2.0,1,1,2,2.0
26522,24015504000390010,2,2,NaN,1.0,1,1,2,2.0
26523,24015504000390011,2,2,NaN,1.0,1,1,2,2.0
26524,24015504000390012,2,2,NaN,1.0,1,2,1,2.0


In [11]:
cols_dummies_ambnac = columnas_dummies(ambnac)
cols_dummies_ambnac

['pa_arealm',
 'pa_areempa',
 'eq_earado',
 'eq_semman',
 'eq_emotoguadana',
 'eq_ebmanual',
 'eq_ebestaciona',
 'eq_ecosgranfigr']

In [12]:
ambnac = convertir_dummies(ambnac, cols_dummies_ambnac, 'str')

In [13]:
for col in cols_dummies_ambnac:
  print(col)
  print(ambnac[col].value_counts())
  print('-'*33)

pa_arealm
0    25487
1     1039
Name: pa_arealm, dtype: int64
---------------------------------
pa_areempa
0    25895
1      631
Name: pa_areempa, dtype: int64
---------------------------------
eq_earado
1.0    4756
0.0    2528
Name: eq_earado, dtype: int64
---------------------------------
eq_semman
0.0    10521
1.0      174
Name: eq_semman, dtype: int64
---------------------------------
eq_emotoguadana
0    23887
1     2639
Name: eq_emotoguadana, dtype: int64
---------------------------------
eq_ebmanual
0    14626
1    11900
Name: eq_ebmanual, dtype: int64
---------------------------------
eq_ebestaciona
0    25097
1     1429
Name: eq_ebestaciona, dtype: int64
---------------------------------
eq_ecosgranfigr
0.0    9527
1.0    1199
Name: eq_ecosgranfigr, dtype: int64
---------------------------------


In [14]:
ambnac

,identificador,pa_arealm,pa_areempa,eq_earado,eq_semman,eq_emotoguadana,eq_ebmanual,eq_ebestaciona,eq_ecosgranfigr
0,1015101086210001,0,0,NaN,0.0,0,0,0,0.0
1,1015101086210002,0,0,NaN,0.0,0,0,0,0.0
2,1015101086210003,0,0,NaN,0.0,0,0,0,0.0
3,1015101086210004,0,0,NaN,0.0,0,0,0,0.0
4,1015101086210005,0,0,NaN,0.0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...
26521,24015504000390009,0,0,NaN,0.0,1,1,0,0.0
26522,24015504000390010,0,0,NaN,1.0,1,1,0,0.0
26523,24015504000390011,0,0,NaN,1.0,1,1,0,0.0
26524,24015504000390012,0,0,NaN,1.0,1,0,1,0.0


In [ ]:
# molestosas = ['eq_earado', 'eq_semman']
# for molestosa in molestosas:
#   ambnac[molestosa] = ambnac[molestosa].astype('Int64')

In [15]:
ambnac.to_csv('limpieza-dummies/ambnac.csv', index=False, sep=',', encoding='utf-8')
ambnac.to_stata('archivos-stata/ambnac.dta', write_index=False)

/usr/local/lib/python3.7/dist-packages/pandas/io/stata.py:611: PossiblePrecisionLoss: 
Column converted from int64 to float64, and some data are outside of the lossless
conversion range. This may result in a loss of precision in the saved data.

  warnings.warn(ws, PossiblePrecisionLoss)


**LIMPIAMOS COVID**

In [ ]:
covid = limpieza_archivo_original(covid)

Se han quitado los espacios en blanco
Se ha establecido el punto como separador de decimales


In [ ]:
cols_dummies_covid = columnas_dummies(covid)
cols_dummies_covid

['ec19_actafec',
 'ec19_dadinsu',
 'ec19_dconmob',
 'ec19_dtraterre',
 'ec19_dofretrab',
 'ec19_pveprfalt',
 'ec19_pveprausc',
 'ec19_accecred',
 'ec19_recayamag',
 'ec19_recayusgr',
 'ec19_recayugc',
 'ec19_recayugadm',
 'ec19_recayugadp',
 'ec19_recayuong',
 'ec19_recayuotro']

In [ ]:
covid = convertir_dummies(covid, cols_dummies_covid)
covid

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,ec19_actafec,ec19_dadinsu,ec19_dconmob,ec19_dtraterre,ec19_dofretrab,ec19_pveprfalt,ec19_pveprausc,ec19_accecred,ec19_recayamag,ec19_recayusgr,ec19_recayugc,ec19_recayugadm,ec19_recayugadp,ec19_recayuong,ec19_recayuotro,ec19_recayudesc
0,17035101244110005,17,3,51,1,24411,5,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,14015301033850001,14,1,53,1,3385,1,1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,17015401048900003,17,1,54,1,4890,3,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,9115404002530004,9,11,54,4,253,4,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,17035101247370001,17,3,51,1,24737,1,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26403,13055003006150008,13,5,50,3,615,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26404,13075002196270005,13,7,50,2,19627,5,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
26405,13125003017700005,13,12,50,3,1770,5,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
26406,21045103021220022,21,4,51,3,2122,22,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
covid.sort_values('Identificador', inplace=True)
covid

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,ec19_actafec,ec19_dadinsu,ec19_dconmob,ec19_dtraterre,ec19_dofretrab,ec19_pveprfalt,ec19_pveprausc,ec19_accecred,ec19_recayamag,ec19_recayusgr,ec19_recayugc,ec19_recayugadm,ec19_recayugadp,ec19_recayuong,ec19_recayuotro,ec19_recayudesc
22663,1015101086210001,1,1,51,1,8621,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21704,1015101086210002,1,1,51,1,8621,2,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
19132,1015101086210003,1,1,51,1,8621,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18834,1015101086210004,1,1,51,1,8621,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21963,1015101086210005,1,1,51,1,8621,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10197,24015504000390009,24,1,55,4,39,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4663,24015504000390010,24,1,55,4,39,10,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
8510,24015504000390011,24,1,55,4,39,11,1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
9185,24015504000390012,24,1,55,4,39,12,1,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
# for col in cols_dummies_covid:
#   print(col)
#   print(covid[col].value_counts())
#   print('-'*33)

In [ ]:
# covid['ec19_actafec'] = covid['ec19_actafec'].astype(str)

In [ ]:
covid.to_csv('limpieza-dummies/covid.csv', index=False, sep=',', encoding='utf-8')
covid.to_stata('archivos-stata/covid.dta', write_index=False)

/usr/local/lib/python3.7/dist-packages/pandas/io/stata.py:611: PossiblePrecisionLoss: 
Column converted from int64 to float64, and some data are outside of the lossless
conversion range. This may result in a loss of precision in the saved data.

  warnings.warn(ws, PossiblePrecisionLoss)


**LIMPIAMOS EMPLEO**

In [ ]:
empleo = limpieza_archivo_original(empleo)

Se han quitado los espacios en blanco
Se ha establecido el punto como separador de decimales


In [ ]:
empleo.sort_values('Identificador', inplace=True)
empleo

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,eu_superficie_ha,eu_k1301,eu_k1302,eu_k1303,eu_k1304,eu_k1305,eu_k1306,eu_k1307,eu_k1308,eu_k1309,eu_k1310,eu_k1311,eu_k1312,eu_trs,eu_trc,eu_tro,eu_tn,fact_exp_fin
0,1015101086210001,1,1,51,1,8621,1,.71,1,1,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,4,64.1680588914427
1,1015101086210002,1,1,51,1,8621,2,1.13,2,1,1,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,4,59.3977383152538
2,1015101086210003,1,1,51,1,8621,3,.62,2,1,1,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,4,47.7636512343916
3,1015101086210004,1,1,51,1,8621,4,1.76,2,1,1,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,4,46.508293796633
4,1015101086210005,1,1,51,1,8621,5,1.36,2,1,1,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,4,60.6417376932933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26656,24015504000390009,24,1,55,4,39,9,10,4,4,NaN,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,4,16.0558804938952
26657,24015504000390010,24,1,55,4,39,10,50,4,4,NaN,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,4,3.16856364118466
26658,24015504000390011,24,1,55,4,39,11,30,1,1,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,4,11.6045216469547
26659,24015504000390012,24,1,55,4,39,12,40,3,3,NaN,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,4,13.2843742318741


In [ ]:
cols_dummies_empleo = columnas_dummies(empleo)
cols_dummies_empleo

[]

In [ ]:
empleo.dtypes

Identificador        int64
ual_prov             int64
ual_cant             int64
ual_parr             int64
ual_estr             int64
ual_segm             int64
al_ncues             int64
eu_superficie_ha    object
eu_k1301             int64
eu_k1302            object
eu_k1303            object
eu_k1304            object
eu_k1305            object
eu_k1306            object
eu_k1307            object
eu_k1308            object
eu_k1309            object
eu_k1310            object
eu_k1311            object
eu_k1312            object
eu_trs              object
eu_trc              object
eu_tro              object
eu_tn                int64
fact_exp_fin        object
dtype: object

In [ ]:
empleo.to_csv('limpieza-dummies/empleo.csv', index=False, sep=',', encoding='utf-8')
empleo = pd.read_csv('limpieza-dummies/empleo.csv')
empleo.to_stata('archivos-stata/empleo.dta', write_index=False)

/usr/local/lib/python3.7/dist-packages/pandas/io/stata.py:611: PossiblePrecisionLoss: 
Column converted from int64 to float64, and some data are outside of the lossless
conversion range. This may result in a loss of precision in the saved data.

  warnings.warn(ws, PossiblePrecisionLoss)


**LIMPIAMOS CARACTERISTICAS**

In [ ]:
caracteristicas

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,cg_vive_predio_pr,cg_terredensm,cg_dista_pre_viv,cg_k100,cg_k101,cg_se_define,cg_k102,cg_segsoccam,cg_accesocomp,cg_accesointer,cg_superficie,cg_unidadmedida,grupo1,edad_rango
0,1015101086210001,1,1,51,1,8621,1,1,1,,1,52,4,1,2,2,2,",71",1,1,3
1,1015101086210002,1,1,51,1,8621,2,2,,1,2,73,4,1,2,2,2,"1,13",1,2,4
2,1015101086210003,1,1,51,1,8621,3,2,,20,2,60,4,1,2,1,1,",62",1,1,3
3,1015101086210004,1,1,51,1,8621,4,1,1,,2,45,4,1,2,2,2,"1,76",1,2,3
4,1015101086210005,1,1,51,1,8621,5,1,1,,2,53,4,1,2,2,2,"1,36",1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36437,24015504000399999,24,1,55,4,39,9999,1,1,,1,99,6,7,2,2,2,6,1,5,6
36438,24015504000550001,24,1,55,4,55,1,2,,99,2,62,4,2,2,2,1,33,1,7,4
36439,24015504000550002,24,1,55,4,55,2,2,,2,1,57,4,3,2,2,2,33,1,7,3
36440,24015504000550003,24,1,55,4,55,3,2,,2,1,99,6,7,2,2,2,400,1,10,6


In [ ]:
caracteristicas.sort_values('Identificador', inplace=True)
caracteristicas = limpieza_archivo_original(caracteristicas)

Se han quitado los espacios en blanco
Se ha establecido el punto como separador de decimales


In [ ]:
cols_dummies_cartcs = columnas_dummies(caracteristicas)
cols_dummies_cartcs

['cg_vive_predio_pr',
 'cg_terredensm',
 'cg_k100',
 'cg_segsoccam',
 'cg_accesocomp',
 'cg_accesointer']

In [ ]:
caracteristicas = convertir_dummies(caracteristicas, cols_dummies_cartcs)
caracteristicas

,Identificador,ual_prov,ual_cant,ual_parr,ual_estr,ual_segm,al_ncues,cg_vive_predio_pr,cg_terredensm,cg_dista_pre_viv,cg_k100,cg_k101,cg_se_define,cg_k102,cg_segsoccam,cg_accesocomp,cg_accesointer,cg_superficie,cg_unidadmedida,grupo1,edad_rango
0,1015101086210001,1,1,51,1,8621,1,1,1.0,NaN,1,52,4,1,0,0,0,.71,1,1,3
1,1015101086210002,1,1,51,1,8621,2,0,NaN,1,0,73,4,1,0,0,0,1.13,1,2,4
2,1015101086210003,1,1,51,1,8621,3,0,NaN,20,0,60,4,1,0,1,1,.62,1,1,3
3,1015101086210004,1,1,51,1,8621,4,1,1.0,NaN,0,45,4,1,0,0,0,1.76,1,2,3
4,1015101086210005,1,1,51,1,8621,5,1,1.0,NaN,0,53,4,1,0,0,0,1.36,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36437,24015504000399999,24,1,55,4,39,9999,1,1.0,NaN,1,99,6,7,0,0,0,6,1,5,6
36438,24015504000550001,24,1,55,4,55,1,0,NaN,99,0,62,4,2,0,0,1,33,1,7,4
36439,24015504000550002,24,1,55,4,55,2,0,NaN,2,1,57,4,3,0,0,0,33,1,7,3
36440,24015504000550003,24,1,55,4,55,3,0,NaN,2,1,99,6,7,0,0,0,400,1,10,6


In [ ]:
caracteristicas.to_csv('limpieza-dummies/caracteristicas.csv', index=False, sep=',', encoding='utf-8')
caracteristicas = pd.read_csv('limpieza-dummies/caracteristicas.csv')
caracteristicas.to_stata('archivos-stata/caracteristicas.dta', write_index=False)

/usr/local/lib/python3.7/dist-packages/pandas/io/stata.py:611: PossiblePrecisionLoss: 
Column converted from int64 to float64, and some data are outside of the lossless
conversion range. This may result in a loss of precision in the saved data.

  warnings.warn(ws, PossiblePrecisionLoss)
